<a href="https://colab.research.google.com/github/shraddhabarke/nl-to-types/blob/master/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install datasets

     |████████████████████████████████| 264 kB 5.1 MB/s 
     |████████████████████████████████| 243 kB 41.0 MB/s 
     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 118 kB 60.2 MB/s 


In [4]:
from datasets import load_dataset

In [6]:
dataset = load_dataset('csv', data_files={'train': 'train-input-types.csv',
                                          'test': 'test-input-types.csv'})

Using custom data configuration default-d9ae5a381171b6e7


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d9ae5a381171b6e7/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


In [7]:
len(dataset)

2

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'code', 'label'],
        num_rows: 1245
    })
    test: Dataset({
        features: ['text', 'code', 'label'],
        num_rows: 254
    })
})


In [9]:
pip install transformers

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 60.6 MB/s 
     |████████████████████████████████| 3.3 MB 47.5 MB/s 
     |████████████████████████████████| 895 kB 43.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.15
    Uninstalling huggingface-hub-0.0.15:
      Successfully uninstalled huggingface-hub-0.0.15


In [10]:
from transformers import AutoTokenizer

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [12]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [13]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
import torch

In [15]:
print(torch.__version__)

1.9.0+cu102


In [16]:
from transformers import AutoModelForSequenceClassification

In [17]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [18]:
from transformers import TrainingArguments

In [19]:
training_args = TrainingArguments("test_trainer")

In [20]:
from transformers import Trainer

In [21]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=tokenized_datasets['train'], eval_dataset=tokenized_datasets['test']
)

In [22]:
dataset["train"][0]

{'code': 'sum(d * 10 ** i for i, d in enumerate(x[::-1]))',
 'label': 0,
 'text': "Concatenate elements of a list 'x' of multiple integers to a single integer"}

In [23]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, code.
***** Running training *****
  Num examples = 1245
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 468


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=468, training_loss=0.29927281436757147, metrics={'train_runtime': 20411.3025, 'train_samples_per_second': 0.183, 'train_steps_per_second': 0.023, 'total_flos': 982746262103040.0, 'train_loss': 0.29927281436757147, 'epoch': 3.0})

In [24]:
import numpy as np

In [25]:
from datasets import load_metric

In [26]:
metric = load_metric("accuracy")

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [27]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, code.
***** Running Evaluation *****
  Num examples = 254
  Batch size = 8


{'eval_accuracy': 0.937007874015748,
 'eval_loss': 0.3183542788028717,
 'eval_runtime': 447.0909,
 'eval_samples_per_second': 0.568,
 'eval_steps_per_second': 0.072}

In [30]:
trainer.predict(test_dataset=tokenized_datasets['test'])

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, code.
***** Running Prediction *****
  Num examples = 254
  Batch size = 8


PredictionOutput(predictions=array([[-2.3370214,  7.074292 , -2.1343775, -1.651087 , -1.0385578],
       [ 5.943722 , -2.2491355, -1.2209442, -1.6980993, -2.0940075],
       [-2.0252335,  6.6426945, -2.2357452, -0.9220215, -1.2381387],
       ...,
       [ 5.986053 , -2.1245527, -1.6776749, -1.3900623, -2.074561 ],
       [-2.1081426,  7.067914 , -2.10493  , -1.972662 , -0.956198 ],
       [ 5.9412947, -2.4339013, -1.0403883, -1.689271 , -2.1088777]],
      dtype=float32), label_ids=array([1, 0, 1, 2, 2, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 0, 2, 0, 0,
       0, 1, 1, 3, 3, 4, 0, 0, 1, 0, 2, 0, 0, 0, 0, 1, 1, 0, 3, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2,
       2, 0, 3, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 3, 0, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1,
       0, 0, 0, 0, 1, 1, 3, 1, 0, 2, 2, 2, 0, 1, 1, 3, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,
 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')